# PlayWright Browser

This toolkit is used to interact with the browser. While other tools (like the `Requests` tools) are fine for static sites, `PlayWright Browser` toolkits let your agent navigate the web and interact with dynamically rendered sites. 

Some tools bundled within the `PlayWright Browser` toolkit include:

- `NavigateTool` (navigate_browser) - navigate to a URL
- `NavigateBackTool` (previous_page) - wait for an element to appear
- `ClickTool` (click_element) - click on an element (specified by selector)
- `ExtractTextTool` (extract_text) - use beautiful soup to extract text from the current web page
- `ExtractHyperlinksTool` (extract_hyperlinks) - use beautiful soup to extract hyperlinks from the current web page
- `GetElementsTool` (get_elements) - select elements by CSS selector
- `CurrentPageTool` (current_page) - get the current page URL


In [ ]:
#!playwright install
#!playwright install-deps
#! apt-get install libatk1.0-0 \
#!         libatk-bridge2.0-0 \
#!         libcups2 \
#!         libxkbcommon0 \
#!         libatspi2.0-0 \
#!         libxcomposite1 \
#!         libxdamage1 \
#!         libxrandr2 \
#!         libpango-1.0-0 \
#!         libcairo2 \
#!         libasound2

In [3]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

Async function to create context and launch browser:

In [4]:
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)

In [5]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

## Instantiating a Browser Toolkit

It's always recommended to instantiate using the `from_browser` method so that the 

In [6]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1091/chrome-linux/chrome> version=120.0.6099.28>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1091/chrome-linux/chrome> version=120.0.6099.28>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1091/chrome-linux/chrome> version=120.0.6099.28>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1091/chrome-linux/chrome> version=120.0.6099.28>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1091/chrome-linux/chrome> version=120.0.6099.28>),
 GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-109

In [7]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

In [11]:
await navigate_tool.arun(
    {"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"}
)

'Navigating to https://web.archive.org/web/20230428131116/https://www.cnn.com/world returned status code 200'

In [12]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun(
    {"selector": ".container__headline", "attributes": ["innerText"]}
)

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [13]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name["current_webpage"].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

## エージェント内での使用

ブラウザツールのいくつかは「StructuredTool」であり、複数の引数を期待するため、これらは「STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION」より古いエージェントとは互換性がありません。

In [14]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [19]:
result = await agent_chain.arun("What are the headers on current_page?")
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "h1, h2, h3, h4, h5, h6",
    "attributes": ["innerText"]
  }
}
```
Observation: [{"innerText": "CNN values your feedback"}, {"innerText": "World"}, {"innerText": "Video"}, {"innerText": "Around the world"}, {"innerText": "EUROPE"}, {"innerText": "MIDDLE EAST "}, {"innerText": "WORLD IN PHOTOS"}, {"innerText": "AFRICA "}, {"innerText": "AMERICAS "}, {"innerText": "ASIA "}, {"innerText": "Featured"}, {"innerText": "TRAVEL "}, {"innerText": "STYLE "}, {"innerText": "WEATHER "}, {"innerText": "More of the latest stories"}, {"innerText": "Something Extra"}, {"innerText": "ARCHITECTURE AROUND THE WORLD "}, {"innerText": "SPORTS "}, {"innerText": "CNN HEROES "}, {"innerText": "Paid Partner Content"}, {"innerText": "Privacy Policy"}, {"innerText": "Cookie List"}]
Thought:The headers on the current page are:

- CNN values your feedback
- World
- Video
- Around the world
- EUROP